Exercice initialement proposé par Élodie Puybareau, remis en forme par Gonzalo Romero-García & Guillaume Tochon

In [ ]:
# 🛑 Import des bibliothèques
import numpy as np
import scipy as sp
import soundfile as sf  # I/O audio
import sounddevice as sd  # Pour écouter des sons depuis jupyter
import matplotlib.pyplot as plt 
from scipy import fft

# Filtrage d'un signal

Le filtrage est une opération **absolument essentielle** en traitement du signal. L'opération de filtrage permet, via la définition d'un *filtre* (logique, non ?) de ne conserver que certaines parties d'un signal et d'en rejeter les autres.

<img src="https://www.datocms-assets.com/53444/1661860595-filtered-signal-graph-2.png?auto=format&fit=max&w=1024" style="width:75%; height:auto;">

L'application la plus répandue du filtrage est la *suppression de bruit* (ou débruitage) d'un signal, comme l'illustre la figure ci-dessus.

Comme on l'a vu lors du chapitre sur la corrélation & convolution, l'opération mathématique qui permet d'effectuer cette opération de filtrage est le **produit de convolution** (et vous êtes renvoyés vers l'exercice correspondant du TP Corrélation & Convolution pour des exemples de filtrage, en pratique, sur des images) : le filtrage d'un signal $x$ par un filtre $h$ s'écrit donc $y(t) = (x \ast h)(t)$. En pratique, $h$ s'appelle la **réponse impulsionnelle** du filtre.

Grâce au théorème de Plancherel, on a $Y$ la transformée de Fourier du signal filtré qui s'écrit $Y(\nu) = X(\nu)H(\nu)$, avec
* $X$ la transformée de Fourier du signal à filter $x$
* $H$ la **fonction de transfert** de $h$. On l'appelle aussi *gabarit du filtre*

En pratique, l'opération de filtrage se définit justement au travers de $H$, donc dans le domaine fréquentiel. En effet, autant il n'est pas évident (voir carrément impossible) de *designer* un filtre directement via sa réponse impulsionnelle, autant il est "relativement" facile de *designer* un gabarit de filtre (par *designer*, on entend ici quelque chose du genre "le filtre se comporte de telle ou telle manière entre telle fréquence et telle fréquence").

Dans cet exercice, vous allez mettre en pratique cette application de filtrage via la transformée de Fourier pour faire du filtrage de son.

## Chargement d'une note de piano

Vous le savez déjà, le son est une onde de pression, qui se propage donc dans l'espace et dans le temps. L'oreille humaine capte ces variations de pression, et grâce à une merveilleuse machinerie (cf le cours d'introduction d'ITSI), convertit ces variations de pression en impulsions électriques, qui sont transmises au cerveau via le nerf auditif. *In fine*, l'opération de conversion effectuée par l'oreille s'apparente elle même à une transformée de Fourier puisque le signal sonore entrant est décomposé en fonction des fréquences qui le composent. En particulier, l'oreille humaine est sensible aux fréquences s'étalant de (grosso modo) 20 Hz à 20 kHz.

En vertu du théorème de Shannon (qui sera vu plus tard dans le cours...), le son est échantillonné avec $44100$ points par seconde :
* donc la fréquence d'échantillonnage est $f_e = 44100 Hz$
* la période d'échantillonnage, c'est-à-dire le laps de temps entre deux points consécutifs est $T_e = \frac{1}{44100}~s$

Numériquement, un signal sonore est donc un signal unidimensionnel (donc un tableau 1D) $x = [x[0],x[1],\dots,]$ ou le $n$ième échantillon $x[n-1]$ correspond en fait au signal sonore au temps $t = n T_e$.

Pour la suite de cet exercice, nous allons travailler avec une note de piano (un C2, de fréquence fondamentale 65.41 Hz), que l'on charge ici avec la bibliothèque `soundfile` (importée comme `sf`)

In [ ]:
# 🛑 chargement de la note
data = sf.read('signaux/note_piano.wav')

### 🛠️ 🚧 👷  À vous de jouer !

Regardez ce qui a été chargé par `sf.read` dans la variable `data`, et déduisez-en le signal `x` et la fréquence d'échantillonnage `fe`.
À partir de là, déterminez la période d'échantillonnage `Te` et le vecteur de temps discret `t` associé au signal `x` (qui doit donc être de la même taille).

<u>Rappels :</u>
* L'accès au nième élément d'un tuple `T` se fait de la même manière que pour un `array`, à savoir `T[n-1]`
* Deux options possibles pour créer un vecteur de temps discret `t`
    *  [`np.arange(tmin,tmax,Te)`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) qui permet de contrôler le pas de temps `Te` entre deux points consécutifs
    *  [`np.linspace(tmin,tmax,N)`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) qui permet de spécifier le nombre de points `N` à échantillonner <br>
    
Vous êtes renvoyés vers le TP de prise en main de la transformée de Fourier si besoin de vous rafraîchir un peu plus la mémoire...

In [ ]:
x = ??? # FIXME
fe = ??? # FIXME
Te = ??? # FIXME
t = ??? # FIXME

In [ ]:
# 🛑 Affichage de la note
def plot_signal(x, t, title=''):
    plt.figure(figsize=(12, 6))
    plt.title(title)
    plt.plot(t, x, "b")
    plt.xlabel('Temps (en secondes)')
    plt.xlim(t.min(),t.max())
    plt.ylim(-1, 1)

plot_signal(x, t, title='Note de piano')

Sortez vos 🎧 et écoutez la note en question :

In [ ]:
# 🛑 Écouter le son
sd.play(x, fe)

### 🛠️ 🚧 👷  À vous de jouer !

Calculez et affichez la transformée de Fourier $X$ de cette note.

<u>Rappel :</u>
* La transformée de Fourier (discrète) se calcule avec [`sp.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html)
* Les fréquences discrètes se calculent avec [`sp.fft.fftfreq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fftfreq.html)
* Vous pouvez aussi utiliser [`sp.fft.fftshift`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fftshift.html) pour remettre les fréquences discrètes et la transformée de Fourier dans l'ordre $[\nu < 0 ; 0 ; \nu > 0]$ (vous êtes renvoyés vers le TP de prise en main de la transformée de Fourier au besoin).

In [ ]:
nu = ??? # FIXME fréquences discrètes
X = ??? # FIXME transformée de Fourier de x

In [ ]:
# 🛑 Affichage de la transformée de Fourier + zoom sur le spectre
def plot_spectrum(X, nu, nu_min=0, nu_max=fe/2):
    # nu_min ; nu_max -> permet de zoomer sur le spectre entre [nu_min, nu_max]
    # par défaut : nu_min = 0 & nu_max = fe/2
    nu_ind = (nu>=nu_min)&(nu<nu_max)
    
    plt.figure(figsize=(15, 5))
    plt.subplot(121)
    plt.title('Module du spectre')
    plt.plot(nu, np.abs(X), 'b')
    plt.xlim(nu.min(),nu.max())
    plt.xlabel('Fréquence (en Hertz)')   
    plt.subplot(122)
    plt.title(f'Zoom de {nu_min} Hz à {nu_max} Hz')
    plt.plot(nu[nu_ind], np.abs(X[nu_ind]), 'b')
    plt.xlim(nu_min,nu_max)
    plt.xlabel('Fréquence (en Hertz)')

# Fréquences qui contrôlent la visualisation
nu_min = 0
nu_max = 3000
plot_spectrum(X, nu, nu_min, nu_max)

## *Design* d'un filtre idéal

Dans ce TP, nous allons considérer 3 grandes familles de filtres :
* les filtres **passe bas**, qui laissent passer toutes les fréquences jusqu'à une certaine fréquence de coupure $\nu_c$. Le gabarit de filtre associé $H_{lp}(\nu)$ est donc tel que $H_{lp}(\nu) = 1$ si $\nu \in [-\nu_c ; \nu_c]$, $H_{lp}(\nu) = 0$ sinon $\Rightarrow$ $H_{lp}$ est une porte (en fréquence) de largeur $2 \nu_c$ (et la réponse impulsionnelle associée $h(t) = \mathcal{F}^{-1}(H_{lp}(\nu))$ est donc un sinus cardinal...)

* les filtres **passe haut**, qui ne laissent passer les fréquences qu'à partir d'une certaine fréquence de coupure $\nu_c$. Le gabarit de filtre associé $H_{hp}(\nu)$ est donc tel que $H_{hp}(\nu) = 0$ si $\nu \in [-\nu_c ; \nu_c]$, $H_{hp}(\nu) = 1$ sinon $\Rightarrow$ $H_{hp}$ est donc le complémentaire d'un filtre passe-bas : $H_{hp} = 1 - H_{lp}$

* les filtres **passe bande**, qui ne laissent passer les fréquences que dans un intervalle $[\nu_{c_1} ; \nu_{c_2}]$. Le gabarit de filtre associé $H_{bp}(\nu)$ est donc tel que $H_{bp}(\nu) = 1$ si $\nu \in [-\nu_{c_2} ; -\nu_{c_1}] \cup [\nu_{c_1} ; \nu_{c_2}]$, $H_{bp}(\nu) = 0$ sinon. $\Rightarrow$ $H_{bp}$ est la différence de deux filtres passe bas, l'un de fréquence de coupure $\nu_{c_2}$ et l'autre de fréquence de coupure $\nu_{c_1}$.

| ![Image 1](https://cpjobling.github.io/eg-247-textbook/_images/ideal_lpf.png) | ![Image 2](https://cpjobling.github.io/eg-247-textbook/_images/ideal_hpf.png) | ![Image 3](https://cpjobling.github.io/eg-247-textbook/_images/bbpmfr.png) |
|:---------------------------------------------:|:---------------------------------------------:|:---------------------------------------------:|
| Illustration d'un gabarit de filtre passe bas | Illustration d'un gabarit de filtre passe haut | Illustration d'un gabarit de filtre passe bande |

<br>
⚠️ Les 3 illustrations précédentes définissent les gabarits de filtres en fonction de leur pulsation $\omega$. Mais vous pouvez mentalement remplacer $\omega$ par $\nu$, ça ne change absolument rien.

In [ ]:
# 🛑 Définition d'un gabarit de filtre
def define_filter(nu, nu_c1=0, nu_c2=fe/2):
    H = np.ones(nu.size)
    H[np.abs(nu) < nu_c1] = 0
    H[np.abs(nu) > nu_c2] = 0
    return H

Dans la fonction précédente, les valeurs par défaut $\nu_{c_1} = 0$ et $\nu_{c_2} = \frac{f_e}{2}$ définissent un filtre $H(\nu) = 1 \ \ \forall \nu$ qui laisse passer toutes les fréquences. En changeant la valeur de $\nu_{c_1}$ ou $\nu_{c_2}$ (ou les deux), on peut donc créer soit un filtre passe bas, soit passe haut, soit passe bande

In [ ]:
# 🛑 Définition d'un filtre passe bas, un filtre passe haut, et un filtre passe bande
H_lp = define_filter(nu, nu_c2 = 5000) # fréquence de coupure passe bas de 5000 Hz
H_hp = define_filter(nu, nu_c1 = 15000) # fréquence de coupure passe haut de 15000 Hz
H_bp = define_filter(nu, nu_c1 = 2500, nu_c2 = 10000) # plage de fréquence [2500 Hz ; 10000 Hz]
# Affichage
plt.figure(figsize=(20,5))
plt.subplot(131)
plt.plot(nu,H_lp,'b')
plt.xlim(nu.min(),nu.max())
plt.title('Gabarit de filtre passe bas')
plt.subplot(132)
plt.plot(nu,H_hp,'b')
plt.xlim(nu.min(),nu.max())
plt.title('Gabarit de filtre passe haut')
plt.subplot(133)
plt.plot(nu,H_bp,'b')
plt.xlim(nu.min(),nu.max())
plt.title('Gabarit de filtre passe bande')
plt.show()

### 🛠️ 🚧 👷  À vous de jouer !

Pour un signal sonore, on considère (grosso modo) que les basses fréquences s'étalent jusqu'à ~1000 Hz, les moyennes fréquences de ~1000 Hz à ~5000 Hz, et les hautes fréquences à partir de ~5000 Hz.<br>
Définissez donc 3 gabarits de filtres adaptés à ces plages de fréquences.

In [ ]:
H_low = ??? # FIXME filtre pour les basses fréquences
H_mid = ??? # FIXME filtre pour les moyennes fréquences
H_high = ??? # FIXME filtre pour les hautes fréquences

## Le filtrage (à proprement parler)

On l'a dit précédemment, l'étape de filtrage à proprement parler consiste à multiplier la transformée de Fourier $X$ du signal à filtrer avec le gabarit du filtre $H$ pour obtenir la transformée de Fourier du signal filtré $Y$ : $$Y(\nu) = X(\nu) H(\nu) $$

Le signal filtré résultant $y$ s'obtient par la suite en prenant la transformée de Fourier inverse de $Y$ :
$$y(t) = \mathcal{F}^{-1}\big(Y(\nu)\big) = \mathcal{F}^{-1}\big(X(\nu) H(\nu)\big) $$ 

### 🛠️ 🚧 👷  À vous de jouer !

Filtrez $X$ avec les 3 gabarits de filtres définis précédemment, et affichez le spectre filtré résultant grâce à la fonction `plot_spectrum` définie précédemment (soyez judicieux sur les définitions de `nu_min` et `nu_max`)<br>
Que remarquez-vous ?

In [ ]:
X_low = ??? # FIXME Spectre basses fréquences
X_mid = ??? # FIXME Spectre moyennes fréquences
X_high = ??? # FIXME Spectre hautes fréquences

# Affichage des spectres filtrés
plot_spectrum(X_low, nu, nu_min = ???, nu_max = ???) # FIXME
plot_spectrum(X_mid, nu, nu_min = ???, nu_max = ???) # FIXME
plot_spectrum(X_high, nu, nu_min = ???, nu_max = ???) # FIXME

Je remarque que ❓❓❓

### 🛠️ 🚧 👷  À vous de jouer !

Inversez les 3 transformées de Fourier obtenues précédemment pour reconstruire dans le domaine temporel les signaux basse fréquence `x_low`, moyenne fréquence `x_mid` et haute fréquence `x_high`, puis affichez ces signaux.

⚠️ Pensez bien à utiliser [`sp.fft.ifftshift`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.ifftshift.html#scipy.fft.ifftshift) **avant** d'inverser la transformée de Fourier avec [`sp.fft.ifft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.ifft.html) si vous aviez utilisé `sp.fft.fftshift` lors du calcul direct de la transformée de Fourier.

⚠️ Même si vous attendez un résultat avec des valeurs réelles après avoir inversé la transformée de Fourier, il est très probable que celui-ci ait un petit résidu complexe. Il conviendra donc d'en prendre la partie réelle avec `np.real`

In [ ]:
x_low = ??? # FIXME reconstruction du signal basse fréquence à partir de X_low
x_mid = ??? # FIXME reconstruction du signal moyenne fréquence à partir de X_mid
x_high = ??? # FIXME reconstruction du signal haute fréquence à partir de X_high

# Affichage
plot_signal(x_low,t,title='signal basse fréquence')
plot_signal(x_mid,t,title='signal moyenne fréquence')
plot_signal(x_high,t,title='signal haute fréquence')

Reprenez vos 🎧 et écoutez le résultat des filtrages basse, moyenne et haute fréquence sur la note de piano.

In [ ]:
# 🛑 Écouter le signal basse fréquence
sd.play(x_low, fe)

In [ ]:
# 🛑 Écouter le signal moyenne fréquence
sd.play(x_mid, fe)

In [ ]:
# 🛑 Écouter le signal haute fréquence
sd.play(x_high, fe)

## Débruitage d'un signal

Vous êtes un producteur musical et venez de faire une prise de son pour enregistrer le prochain jingle de la grande entreprise pour laquelle vous travaillez (dont nous ne pouvons dévoiler le nom...). Malheureusement, après avoir réécouté l'enregistrement, vous vous rendez compte que vous aviez laissé le téléphone décroché et qu'il y a un son parasite. Les instrumentistes sont partis et vous ne pouvez plus faire une autre prise.

Heureusement, vous vous souvenez (vaguement) de vos cours de traitement du signal quand vous étiez étudiant à l'EPITA et vous pensez pouvoir filtrer ce son.

In [ ]:
# 🛑 Lecture et affichage du fichier audio
data = sf.read('signaux/mistery_signal.wav')
x = data[0]
fe = data[1]
Te = 1/fe
t = np.arange(0,x.size)*Te

In [ ]:
# 🛑 Affichage de la note
plot_signal(x, t, title='Enregistrement corrompu')

In [ ]:
# 🛑 Écouter le son
sd.play(x,fe)

### 🛠️ 🚧 👷  À vous de jouer !

1. Calculez et affichez la transformée de Fourier du signal bruité.
2. Identifiez la fréquence parasite (elle est facilement reconnaissable) et zoomez dessus en jouant avec les variables `nu_min` et `nu_max` de la fonction `plot_spectrum`
3. Construisez un filtre *réjecteur de bande* qui laisse passer toutes les fréquences sauf pour un intervalle donné.
En pratique, un filtre réjecteur de bande de fréquence $[\nu_{c_1} ; \nu_{c_2}]$ se construit comme le complémentaire d'un filtre passe bande : pour filtrer uniquement ce qui se trouve dans l'intervalle $[\nu_{c_1} ; \nu_{c_2}]$, le gabarit se définit en fréquence comme $1 - H_{bp}$ avec $\nu_{c_1}, \nu_{c_2}$ les fréquences de coupure du filtre passe bande $H_{bp}$.
4. Filtrez la transformée de Fourier du signal bruité, puis reconstruisez le signal filtré dans le domaine temporel en inversant la transformée de Fourier après filtrage
5. Pour finir, affichez le signal débruité.

In [ ]:
# calcul et affichage de la transformée de Fourier
X = ??? # FIXME transformée de Fourier du signal bruité
nu = ??? # FIXME fréquences discrètes associées
plot_spectrum(X, nu, nu_min = ???, nu_max = ???) # FIXME affichage et zoom de la transformée de Fourier

# design d'un filtre rejecteur de bande
H_bs = ??? # FIXME gabarit d'un fitre rejecteur de bande

# filtrage et inversion de la transformée de Fourier
X_filt = ??? # FIXME filtrage de la transformée de Fourier
x_filt = ??? # FIXME inversion / reconstruction dans le domaine temporel

# affichage du signal filtré
plot_signal(x_filt, t, title='Enregistrement débruité')

Écoutez maintenant le signal débruité. Votre fitrage a t-il été efficace ?

In [ ]:
# 🛑 Écouter le son
sd.play(x_filt,fe)

# Bravo !
Vous en avez fini avec cet exercice sur le filtrage d'un signal grâce à la transformée de Fourier. Vous pouvez donc maintenant attaquer l'un des deux exercices suivant (si pas déjà fini) :
* [Traitement d'un signal d'ECG cardiaque](TP3_Fourier_application_ECG.ipynb)
* [Application du théorème de Plancherel pour la complexité de calcul de la convolution](TP3_Fourier_application_Plancherel.ipynb)